In [ ]:
import numpy as np
import os, sys
from tqdm import tqdm
import tifffile as tiff
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd
import imageio

## pseudo code for tiff file creation

In [ ]:
# Imports 
import tiff??? 


def create_3d_array(x_size, y_size, z_size):
    array_3d = [[ ['#' for col in range(x_size)] for col in range(y_size)] for row in range(z_size)]
    return array_3d
    
def load_point_cloud(path_to_xyz_file):
    xyz_data = list()
    f = open(path_to_xyz_file)
    for line in file:
        data = line.split(' ')
        x = float(data[0])
        y = float(data[1])
        z = float(data[2])
        
        point = [x, y, z]
        xyz_data.append(point)
    f.close()
    return xyz_data

# Load the point cloud from file 
point_cloud = load_point_cloud(path_to_xyz_file)


# Get the dimensions (bounding box) of the point cloud 
x_min = 1e32
y_min = 1e32
z_min = 1e32
x_max = -1e32
y_max = -1e32
z_max = -1e32

for point in point_cloud:
    if p[0] < x_min: x_min = p[0]
    if p[1] < y_min: y_min = p[1]
    if p[2] < z_min: z_min = p[2]
    
    if p[0] > x_max: x_max = p[0]
    if p[1] > y_max: y_max = p[1]
    if p[2] > z_max: z_max = p[2]

# Dimensions in nano-meters 
width = x_max - x_min
height = y_max - y_min
depth = z_max - z_min

# Define the resolution of the grid 
resolution = 5

grid_width = resolution * width
grid_height = resolution * height
grid_depth = resolution * depth


# Construct a 3D array 
array_3d = create_3d_array(grid_width, grid_height, grid_depth)

# Register the points in the grid 
for point in point_cloud:

     # Get the corresponding voxel for the point cloud coordinates 
     voxel_x = ?
     voxel_y = ?
     voxel_z = ?
     
     array_3d[voxel_x][voxel_y][voxel_z] += 1
     

# Save the array into a tiff file 
???

---

In [ ]:
path = './single/single_chr10_2000/Chromosome_block0.xyz'
xyz = open(path, 'r')
xyz

In [ ]:
xyz.readline()

In [ ]:
xyz.readline()

### read lines of xyz file

In [ ]:
# PACKAGE AS FUNCTION
def read_xyz(path):
    xyz = open(path, 'r')

    # FIRST LINE IS THE NUMBER OF ATOMS
    n_atoms = int(xyz.readline())

    # ITERATE THROUGH LINES IN FILE TO GET COORDINATES
    coordinates = []
    for line in xyz:
        x, y, z = line.split()
        coordinates.append([float(x), float(y), float(z)])
    coordinates = np.array(coordinates)
    xyz.close()

    return n_atoms, coordinates

In [ ]:
def read_xyz_folder(folder_path):
    n_frames = len(os.listdir(folder_path))
    frames = os.listdir(folder_path)[1:]

    frame_atoms, frame_coordinates = [], []
    for frame in tqdm(frames[1:-3]):
        path = os.path.join(folder_path, frame)
        atoms, coordinates = read_xyz(path)
        frame_atoms.append(atoms)
        frame_coordinates.append(coordinates)
    frame_atoms = np.array(frame_atoms)
    frame_coordinates = np.array(frame_coordinates)

    print('number of frames = {}'.format(frame_atoms.shape))
    print('frame coordinates shape = {}'.format(frame_coordinates.shape))

    return frame_atoms, frame_coordinates

In [ ]:
# READ ONE FRAME ONLY
atoms, coordinates = read_xyz(path)
coordinates, coordinates.shape

In [ ]:
# READ ALL FRAMES STACKED
xyz_folder_path = './single/single_op_chr10_xyz'
frame_atoms, frame_coordinates = read_xyz_folder(xyz_folder_path)

In [ ]:
# READ ONE FRAME ONLY
one_frame = frame_coordinates[0]
type(one_frame)

In [ ]:
X = coordinates.T[0]
Y = coordinates.T[1]
Z = coordinates.T[2]
X.shape

In [ ]:
X = coordinates.T[0].reshape(-1,1)
Y = coordinates.T[1].reshape(-1,1)
Z = coordinates.T[2].reshape(-1,1)
X.shape

### testing tifffile approach

In [ ]:
tiff.imwrite(
    'compressed_xyz.ome.tif', 
    one_frame, 
    imagej = True, 
    resolution = (1./0.22, 1./0.22), 
    metadata = {'sapcing': 0.72, 'unit': 'um', 'axes': 'XYZ'}, 
    compression = 'zlib'
)

In [ ]:
Data_folder = "/Input" #@param {type:"string"}
Result_folder = "/Output" #@param {type:"string"}
name = os.path.splitext(image)
for image in os.listdir(Data_folder):
        prediction_stack_16 = imread(Data_folder+"/"+image)   
        os.chdir(Result_folder)
        # tifffile.imwrite(str(name[0])+"_compressed.tif", prediction_stack_16, metadata={'axes': 'XYZC'}, compression ='zlib')
        tifffile.imwrite("_compressed.ome.tif", prediction_stack_16,imagej=True, resolution=(1./0.22, 1./0.22), metadata={'spacing': 0.72, 'unit': 'um', 'axes': 'ZCYX'}, compression ='zlib')

### testing mesh grid approach

In [ ]:
gridX, gridY = np.meshgrid(X, Y)

gridZ1 =  np.abs(((gridX - 10) ** 2 + (gridY - 10) ** 2) / 1 ** 2)
gridZ2 =  np.abs(((gridX + 10) ** 2 + (gridY + 10) ** 2) / 2.5 ** 2)
gridZ =  (gridZ1 - gridZ2)

plt.imshow(gridZ)
plt.title("Temperature")
plt.show()

In [ ]:
from rasterio.transform import Affine
import rasterio as rio

res = (X[-1] - X[0]) / 240.0
transform = Affine.translation(X[0] - res / 2, Y[0] - res / 2) * Affine.scale(res, res)

# open in 'write' mode, unpack profile info to dst
with rio.open(
   "../temp/new_raster.tif",
   "w",
   driver = "GTiff",         # output file type
   height = gridZ.shape[0],      # shape of array
   width = gridZ.shape[1],
   count = 1,                # number of bands
   dtype = gridZ.dtype,          # output datatype
   crs = "+proj=latlong",    # CRS
   transform = transform,    # location and resolution of upper left cell
) as dst:
   # check for number of bands
   if dst.count == 1:
       # write single band
       dst.write(gridZ, 1)
   else:
       # write each band individually
       for band in range(len(gridZ)):
           # write data, band # (starting from 1)
           dst.write(gridZ[band], band + 1)

### testing geopandas approach

In [ ]:
df = pd.DataFrame(coordinates, columns=['X', 'Y', 'Z'])
df.head()

In [ ]:
# Create a Shapely points from the coordinate-tuple list
ply_coord = [Point(x, y) for x, y, z in zip(df.X, df.Y, df.Z)]
poly = gpd.GeoDataFrame(df, geometry=ply_coord)
poly.plot()

&uarr; only shows 2D plot  
### testing matplotlib approach

In [ ]:
ax = plt.figure(figsize=(15,15)).add_subplot(projection='3d')
# ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('z')
# ax.contour(X, Y, Z)
ax.plot_wireframe(X, Y, Z, rstride=10, cstride=10)

plt.show()

### MinMax scaling approach

In [ ]:
copy = np.copy(frame_coordinates)
print(copy[1995,10,2], frame_coordinates[1995,10,2])

In [ ]:
#Draft


no_of_frames = len(copy)

for frame in frame_coordinates:
    for bead in frame :
        for dimension in bead:
            print(dimension)

In [ ]:
# Shifitng and scaling data (Binning) 

from sklearn.preprocessing import MinMaxScaler


def scale_data(data):
    
    scaler = MinMaxScaler(feature_range=(0,1000))
    scaler.fit(data)
    scaled_data = scaler.transform(data)
    scaled_data = np.around(scaled_data)
    scaled_data = scaled_data.astype(int)
    return scaled_data

In [ ]:
frames = copy
frames[:,:,0] = scale_data(copy[:,:,0])
frames[:,:,1] = scale_data(copy[:,:,1])
frames[:,:,2] = scale_data(copy[:,:,2])

In [ ]:
# Convert a frame to an image
frames.shape

In [ ]:
# How many unique values are there in x-axis in a single arbitrary frame? (Out of 1000)

len(np.unique(frames[80,:,0]))

&uarr; now the coordinates are scaled from 0 to 1000

frames

### testing cndbTools approach

In [ ]:
from OpenMiChroM.ChromDynamics import MiChroM
from OpenMiChroM.CndbTools import cndbTools

In [ ]:
cndbTools = cndbTools()
chr10_traj = cndbTools.load('./single/single_chr10_2000/traj_chr10_2000_0.cndb')
print(chr10_traj)
# EXTRACT LOCI XYZ POSITIONS OVER FRAMES
chr10_xyz = cndbTools.xyz(frames=[1,2000,1], beadSelection='all', XYZ=[0,1,2])

#### `cnbdTools.xyz()` &rarr; get the selected beads' 3D position from a *cndb* or *ndb* for multiple frames  
* write .xyz file
* convert to .tiff

In [ ]:
print(chr10_xyz.shape)

In [ ]:
with open('./single/stack_output/xyz_numpy.npy', 'wb') as f:
    np.save(f, chr10_xyz)

a = []
with open('./single/stack_output/xyz_numpy.npy', 'rb') as f:
    a = np.load(f)
a.shape

In [ ]:
with open('./single/stack_output/xyz_numpy.tiff', 'wb') as f:
    np.save(f, chr10_xyz)

a = []
with open('./single/stack_output/xyz_numpy.tiff', 'rb') as f:
    a = np.load(f)
a.shape

&uarr; tiff file saved using numpy is corrupt or empty

In [ ]:
imageio.imwrite('./single/stack_output/xyz_imageio.tiff', chr10_xyz)
a = imageio.imread('./single/stack_output/xyz_imageio.tiff')
a.shape